# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [6]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [7]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('GEMINI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = "gemini-1.5-flash"
openai = OpenAI(api_key=api_key,    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [9]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [10]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"


link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"


link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""


print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [18]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [19]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/Datou1111/shou_xin',
 '/IamCreateAI/Ruyi-Mini-7B',
 '/CohereForAI/c4ai-command-r7b-12-2024',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/answerdotai/ModernBERT-base',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/HuggingFaceH4/blogpost-scaling-test-time-compute',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/aiqtech/kolaw',
 '/datasets/amphora/QwQ-LongCoT-130K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer'

In [24]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [25]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [26]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
Datou1111/shou_xin
Updated
11 days ago
•
26.6k
•
661
IamCreateAI/Ruyi-Mini-7B
Updated
4 days ago
•
7.23k
•
332
CohereForAI/c4ai-command-r7b-12-2024
Updated
about 4 hours ago
•
3.9k
•
290
meta-llama/Llama-3.3-70B-Instruct
Updated
10 days ago
•
229k
•
1.2k
answerdotai/ModernBERT-base
Updated
about 22 hours ago
•
1.83k
•
211
Browse 400k+ models
Spaces
Running
on
Zero
1.75k
🏢
TRELLIS
Scalable and Versatile 3D Generation fr

In [35]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [36]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [37]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nDatou1111/shou_xin\nUpdated\n11 days ago\n•\n26.6k\n•\n661\nIamCreateAI/Ruyi-Mini-7B\nUpdated\n4 days ago\n•\n7.23k\n•\n332\nCohereForAI/c4ai-command-r7b-12-2024\nUpdated\nabout 4 hours ago\n•\n3.9k\n•\n290\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n10 days ago\n•\n229k\n•\n1.2k\nanswerdotai/ModernBERT-base\nUpdated\nabout 22 hours ago\n•\n1.83k\n•\n211\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.75k\n🏢\nTRELLIS\nScalable a

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com'}, {'type': 'models page', 'url': 'https://huggingface.com/models'}, {'type': 'datasets page', 'url': 'https://huggingface.com/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.com/spaces'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face: Where AI Gets a Hug (and a Job!)

**(A Brochure for the Curious, the Clever, and the Capitalists)**

![Hugging Face Logo](huggingface_logo.png) *(Insert a fun, friendly logo here)*

**Are you tired of cold, hard AI?**  Then get ready for some warm fuzzies! At Hugging Face, we're building the *friendliest* AI community on the planet.  We're not just about algorithms and APIs; we're about collaboration, open-source awesomeness, and making AI accessible to everyone – from coding kittens to seasoned AI gurus.

**What we do:** We're the ultimate playground for machine learning! Think of us as the ultimate online community center for AI:

* **Models:**  Over 400,000 models, ready to be played with, improved, or even just admired from afar. (Some even respond to "hug" – we haven't tested that thoroughly, though.)
* **Datasets:**  Need data? We've got a mountain (or rather, a 100,000+ dataset) of it. Perfect for training your AI, or just plain satisfying your data curiosity.
* **Spaces:**  Show off your AI creations!  Think of it as an AI art gallery, but way cooler.  (And no velvet ropes!)


**Who we love:**

* **Customers:**  Everyone from scrappy startups (who get extra hugs) to multinational corporations (who also get hugs, but maybe slightly less enthusiastic ones).  We cater to all sizes and levels of AI expertise.
* **Investors:**  Want to back the future of AI? We're not just building a platform; we're building a community.  Invest in hugs, and you invest in the future.
* **Recruits:**  If you're a passionate, talented, and fun-loving individual, we want YOU! We offer a relaxed, collaborative work environment where your creativity will be celebrated (and yes, even hugged).  We're always looking for talented engineers, researchers, and community managers to join our growing team. Check out our careers page for openings!


**Our Culture:** Think a friendly neighborhood pub, crossed with a high-tech laboratory, with extra hugs. (We're serious about the hugs). We value collaboration, open-source contributions and a healthy work-life balance.  We also have an unlimited supply of coffee, because you can't be fuzzy-headed when you're building the future of AI.

**Our Open Source Contributions:** We believe in the power of shared knowledge, so we’ve created some awesome tools including: Transformers, Diffusers, Safetensors and more!  

**Join the Hugging Face Revolution!**

Visit us at [www.huggingface.co](www.huggingface.co)

**(P.S. We may or may not have a dedicated hugging department.  It's a work in progress.)**


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [40]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face: Where AI Gets a Hug (and a Job!)

**(A Brochure for the Curious, the Bold, and the Algorithmically Inclined)**

**(Image: A friendly cartoon robot giving a thumbs-up, maybe with a little heart emoji)**


**Inside Scoop:** Forget stuffy labs – Hugging Face is the *cool* kid on the AI block. We're the online playground where over 50,000 organizations (including giants like Google and Meta!) collaborate on models, datasets, and applications. Think of us as the ultimate AI social network, but with way more computational power.


**For Customers:**

* **Models galore:** Need a model to generate killer poetry?  We've got it. A model to sort your sock drawer?  Possibly.  (We have 400,000+ models, so the chances are good!)
* **Datasets to drool over:** We have over 100,000 datasets, ready for your data-hungry algorithms.
* **Spaces to play:** Show off your AI creations in our interactive Spaces!  Think of it as your personal AI portfolio, but cooler.


**For Investors:**

* **Explosive growth:** Our community is exploding, and so is our user base. It's a future full of hugs (and potential profits!).
* **Open-source champion:** We're committed to open-source, driving innovation and making AI accessible to everyone. This translates into a loyal community and a wider user base.


**For Job Seekers:**

* **Be part of the hug:** We're looking for brilliant minds to join our team. Whether you're a seasoned AI guru or a coding newbie with a passion for all things AI, we want to hear from you.
* **Flexible & Fun:** We believe in work-life balance (and maybe even work-life *integration*, if you're into that kind of thing!) Our culture is dynamic, inclusive and collaborative.
* **Impactful work:** Make a real difference in the world of AI.  We're not just building technology; we're building the future, one hug at a time.


**Ready to join the revolution?**

Visit [huggingface.co](huggingface.co) today!

**(Image: A collage of happy people working on laptops, with maybe a robot or two hanging out in the background)**


**(Small print at the bottom):  *While we can't guarantee a robot will greet you at the office, we can guarantee an awesome team and challenging work.*)


In [34]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face: Shaping the Future of AI

**A Collaborative Platform for Machine Learning**

**(Brochure for Prospective Customers, Investors, and Recruits)**


**For Customers:**

Hugging Face is the leading platform where the machine learning community collaborates to build the future of AI. We offer a comprehensive ecosystem for creating, discovering, and sharing:

* **400,000+ Models:** Access a vast library of pre-trained models for various tasks, including text, image, video, audio, and 3D generation.  Find cutting-edge models like Llama-3.3-70B-Instruct and Ruyi-Mini-7B, updated frequently.
* **100,000+ Datasets:** Utilize a rich collection of datasets to fuel your AI projects.  Explore datasets spanning computer vision, audio, and natural language processing.
* **150,000+ Spaces:** Discover and deploy applications built by the community, showcasing innovative uses of machine learning.  Examples include 3D generation from images and style transfer.

**Accelerate your ML workflow:** Our platform streamlines your development process with optimized inference endpoints and GPU-powered Spaces applications, starting at just $0.60/hour for GPU compute.

**Enterprise Solutions:**  For organizations requiring enterprise-grade security and dedicated support, our enterprise offering provides single sign-on, priority support, audit logs, and more, starting at $20/user/month.  Over 50,000 organizations, including major players like Meta, Amazon Web Services, Google, and Microsoft, already leverage the power of Hugging Face.


**For Investors:**

Hugging Face is at the forefront of the rapidly expanding AI industry.  Our open-source contributions, coupled with a thriving community and robust commercial platform, position us for significant growth.  Key investments include:

* **A vibrant, collaborative community:**  Fueled by open-source contributions, our platform attracts and retains top talent in the AI field.
* **A scalable and versatile platform:**  Our infrastructure handles the demands of a rapidly growing community and a vast library of models and datasets.
* **Robust commercial offerings:**  We provide valuable services to individuals and enterprises, driving sustainable revenue growth.

Hugging Face is building the future of AI, one collaboration at a time.


**For Recruits:**

Join a passionate team of engineers, researchers, and community managers shaping the future of AI. We offer:

* **A collaborative and innovative environment:**  Work alongside leading experts in the field and contribute to cutting-edge open-source projects.
* **Impactful work:**  Your contributions will directly affect the global AI community and influence the development of transformative technologies.
* **Opportunities for growth:**  We offer a dynamic and supportive environment with numerous opportunities for professional development.


**Visit our website at [Hugging Face Website Address] to learn more and join our community!**


**(Include logos and visual elements as appropriate in the final brochure design.)**


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>